*This notebook shows an example of making safety hazard identification based on sentence similarity and then output an excel table for evaluation*

In [ ]:
import json

# read result from json file
def read_result_from_json(json_path):

    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

result = read_result_from_json('../processed_result.json')
result

In [ ]:
from tqdm import tqdm
# select images contain "brick" word in caption
def select_image_with_word(result, words, number=100):
    images = []
    for img in tqdm(result):
        if len(images) >= number:
            break
        # get all captions in image
        captions = []
        for group in img['bbox']:
            for bbox in group:
                captions.append(bbox['cap'])
        # check if all the list of word in any one of captions
        if all(word in ' '.join(captions) for word in words):
            images.append(img)

    return images

brick_images = select_image_with_word(result, ['brick'])
brick_images

In [ ]:
from semantic_similarity import *
import pandas as pd
from PIL import Image
import os

nlp = spacy.load('en_use_md')

rules = [
    'worker should wear a safety helmet',
    'worker should wear gloves'
]
# compare each group of caption in each image with a set of rules using semantic similarity
def compare_caption_with_rules(images, rules, nlp, sim_threshold=0.6):
    # parse the results into a pandas dataframe
    df = pd.DataFrame(columns=['image_file', 'image', 'group', 'captions', 'rule', 'similarities', 'max_similarity', 'unsafe', 'rule_violated'])
    for img in tqdm(images):
        for g, group in enumerate(img['bbox']):
            # get all captions in this group as a list of string
            captions = [bbox['cap'] for bbox in group]
            nlp_cap = spacy_parse_list(captions, nlp)
            nlp_rule = spacy_parse_list(rules, nlp)
            similarities = get_similarity_matrix(nlp_cap, nlp_rule)
            similairty_list = similarities.max(axis=0).tolist()
            # check if the max similarity is greater than the threshold and get the index of the rule
            not_compliance = [sim < sim_threshold for sim in similairty_list]
            index_violated = [i for i, x in enumerate(not_compliance) if x]
            # get the rule violated
            rule_violated = [rules[i] for i in index_violated]
            # write to dataframe
            df = df.append({
                'image_file': img['img'],
                # get only the file name
                'image': os.path.basename(img['img']),
                'group': g,
                'captions': captions,
                'rule': rules,
                'similarities': similarities.tolist(),
                'max_similarity': similairty_list,
                'unsafe': len(index_violated)>0,
                'rule_violated': rule_violated
            }, ignore_index=True)
            # embed the image file into the dataframe


    return df


In [ ]:
df = compare_caption_with_rules(brick_images, rules, nlp, sim_threshold=0.4)

In [ ]:
# group by image
df.groupby('image_file').sum()
df

In [ ]:
# save to excel file
df.to_excel('height_images.xlsx', index=False)